In [1]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from IPython.display import display, HTML
%matplotlib inline

def parse_if_number(s):
    try: return float(s)
    except: return True if s=="true" else False if s=="false" else s if s else None

def parse_ndarray(s):
    return np.fromstring(s, sep=' ') if s else None

def get_file_name(name):
    return name.replace(':', '-')

In [2]:
inputFile = 'lowtime.csv'
repetitionsCount = -1 # -1 = auto-detect

# Filter scalars
scalarsFilter = ['Floorplan.userCount']
# Filter vectors
vectorsFilter = ['Floorplan.coveredUsers:vector']
# Percentiles
percentiles = [0.9, 0.95, 0.98, 0.99]

intPercentiles = [int(i*100) for i in percentiles]
broadcastTimes = []
for intPercentile in intPercentiles:
    broadcastTimes.append(('broadcastTime' + str(intPercentile), 'Broadcast time needed to reach the ' + str(intPercentile) + 'th percentile of the coverage'))

In [3]:
df = pd.read_csv('exported_data/' + inputFile, converters = {
    'attrvalue': parse_if_number,
    'binedges': parse_ndarray,
    'binvalues': parse_ndarray,
    'vectime': parse_ndarray,
    'vecvalue': parse_ndarray,
})

In [4]:
if repetitionsCount <= 0: # auto-detect
    repetitionsCount = int(df[df.attrname == 'repetition']['attrvalue'].max()) + 1
print('Repetitions:', repetitionsCount)

display(HTML("<style>div.output_scroll { height: auto; max-height: 48em; }</style>"))
pd.set_option('display.max_rows', 1000)

Repetitions: 30


In [5]:
scalars = df[(df.type == 'scalar') | ((df.type == 'itervar') & (df.attrname != 'TO')) | ((df.type == 'param') & (df.attrname == 'Floorplan.userCount')) | ((df.type == 'runattr') & (df.attrname == 'repetition'))]
scalars = scalars.assign(qname = scalars.attrname.combine_first(scalars.module + '.' + scalars.name))
for index, row in scalars[scalars.type == 'itervar'].iterrows():
    val = scalars.loc[index, 'attrvalue']
    if isinstance(val, str) and not all(c.isdigit() for c in val):
        scalars.loc[index, 'attrvalue'] = eval(val)
scalars.value = scalars.value.combine_first(scalars.attrvalue.astype('float64'))
scalars_wide = scalars.pivot_table(index=['run'], columns='qname', values='value')
scalars_wide.sort_values(['X', 'Y', 'repetition'], inplace=True)
for index, row in scalars_wide.iterrows():
    if row['X'] == 0 and row['Y'] == 0:
        pos = 'corner'
    elif row['X'] == 0 or row['Y'] == 0:
        pos = 'border'
    else:
        pos = 'center'
    scalars_wide.loc[index, 'position'] = pos
scalars_wide = scalars_wide[['position', 'repetition', *scalarsFilter]]

In [6]:
vectors = df[df.type == 'vector']
vectors = vectors.assign(qname = vectors.module + '.' + vectors.name)
for index in scalars_wide.index:
    r = index
    pos = scalars_wide.loc[index, 'position']
    rep = scalars_wide.loc[index, 'repetition']
    vectors.loc[vectors.run == r, 'position'] = pos
    vectors.loc[vectors.run == r, 'repetition'] = rep
vectors = vectors[vectors.qname.isin(vectorsFilter)]
vectors.sort_values(['position', 'repetition', 'qname'], inplace=True)
vectors = vectors[['position', 'repetition', 'qname', 'vectime', 'vecvalue']]

In [7]:
def get_percentile(percentile, vectime, vecvalue, totalvalue):
    tofind = percentile * totalvalue
    idx = 0
    csum = vecvalue.cumsum()
    for value in csum:
        if value >= tofind:
            return vectime[idx]
        idx += 1
    return math.inf

for index, row in vectors.iterrows():
    for bt, percentile in zip(broadcastTimes, percentiles):
        btname = bt[0]
        pos = row['position']
        rep = row['repetition']
        total = scalars_wide[(scalars_wide['position'] == pos) & (scalars_wide['repetition'] == rep)]['Floorplan.userCount'].values[0] - 1
        value = get_percentile(percentile, row['vectime'], row['vecvalue'], total)
        scalars_wide.loc[(scalars_wide['position'] == pos) & (scalars_wide['repetition'] == rep), btname] = value
scalars_wide

qname,position,repetition,Floorplan.userCount,broadcastTime90,broadcastTime95,broadcastTime98,broadcastTime99
run,,,,,,,
StartNodePositionLowDensityTime-0-20210614-12:44:24-28477,corner,0.0,1250.0,56.0,62.0,65.0,71.0
StartNodePositionLowDensityTime-1-20210614-12:44:40-28477,corner,1.0,1250.0,57.0,60.0,65.0,70.0
StartNodePositionLowDensityTime-2-20210614-12:44:56-28477,corner,2.0,1250.0,58.0,61.0,64.0,68.0
StartNodePositionLowDensityTime-3-20210614-12:45:12-28477,corner,3.0,1250.0,57.0,62.0,67.0,75.0
StartNodePositionLowDensityTime-4-20210614-12:45:28-28477,corner,4.0,1250.0,53.0,57.0,61.0,66.0
StartNodePositionLowDensityTime-5-20210614-12:44:24-28478,corner,5.0,1250.0,59.0,64.0,65.0,67.0
StartNodePositionLowDensityTime-6-20210614-12:44:40-28478,corner,6.0,1250.0,54.0,57.0,63.0,65.0
StartNodePositionLowDensityTime-7-20210614-12:44:56-28478,corner,7.0,1250.0,58.0,63.0,66.0,73.0
StartNodePositionLowDensityTime-8-20210614-12:45:12-28478,corner,8.0,1250.0,57.0,60.0,67.0,70.0


In [8]:
for pos in ['center', 'border', 'corner']:
    print('Stats for position', pos)
    display(scalars_wide[scalars_wide['position'] == pos].describe()[[item[0] for item in broadcastTimes]])

Stats for position center


qname,broadcastTime90,broadcastTime95,broadcastTime98,broadcastTime99
count,30.000000,30.000000,30.000000,30.00
mean,29.333333,31.733333,35.166667,inf
std,0.958927,1.048261,1.533158,NaN
min,28.000000,29.000000,31.000000,33.00
25%,29.000000,31.000000,35.000000,36.00
50%,29.000000,32.000000,35.000000,38.00
75%,30.000000,32.750000,36.000000,38.75
max,31.000000,33.000000,40.000000,inf


Stats for position border


qname,broadcastTime90,broadcastTime95,broadcastTime98,broadcastTime99
count,30.000000,30.000000,30.000000,30.00
mean,47.166667,50.066667,53.700000,inf
std,1.510500,1.638614,1.914554,NaN
min,44.000000,46.000000,50.000000,53.00
25%,46.000000,49.000000,52.250000,55.00
50%,47.000000,50.000000,53.500000,57.00
75%,48.000000,51.000000,55.000000,58.75
max,50.000000,53.000000,58.000000,inf


Stats for position corner


qname,broadcastTime90,broadcastTime95,broadcastTime98,broadcastTime99
count,30.00000,30.000000,30.000000,30.0
mean,56.50000,60.733333,65.100000,inf
std,2.19325,2.420221,2.186952,NaN
min,53.00000,57.000000,61.000000,65.0
25%,55.00000,59.000000,64.000000,68.0
50%,56.50000,60.500000,65.000000,70.0
75%,58.00000,62.000000,67.000000,71.0
max,62.00000,67.000000,70.000000,inf


### Rerun this notebook

To rerun this notebook, you can:
- just rerun the simulations with the corresponding configuration: `./simulate.sh -s StartNode -c StartNodePositionLowDensityTime -o lowtime` (you will get slighly different results)
- download our datasets from `https://drive.google.com/file/d/1ZFRV2DecoTvax9lngEsuPPw8Cz1DXvLc/view?usp=sharing` (login with UNIPI institutional account)
- use our seed to rerun the simulations. Add `seed-set = ${runnumber}28479` to the configuration